In [23]:
import random
import csv
from datetime import datetime, timedelta

def generate_random_date():
    start_date = datetime(2024, 3, 1)
    end_date = datetime(2024, 3, 23)
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

def generate_user_activity_data(num_entries=100):
    data = []
    for _ in range(num_entries):
        user_id = random.randint(10000, 99999)
        last_visit = generate_random_date().strftime('%Y-%m-%d')
        time_spent = round(random.uniform(1, 60), 2)  # Time spent in minutes
        pages_viewed = random.randint(1, 10)  # Number of pages viewed
        interaction_score = round(random.uniform(0, 1), 2)  # Score between 0 and 1
        data.append([user_id, last_visit, time_spent, pages_viewed, interaction_score])
    return data

def save_to_csv(filename, data):
    header = ["User ID", "Last Visit", "Time Spent (minutes)", "Pages Viewed", "Interaction Score"]
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(data)

def load_user_data(filename):
    users = []
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            row['User ID'] = int(row['User ID'])
            row['Time Spent (minutes)'] = float(row['Time Spent (minutes)'])
            row['Pages Viewed'] = int(row['Pages Viewed'])
            row['Interaction Score'] = float(row['Interaction Score'])
            row['Last Visit'] = datetime.strptime(row['Last Visit'], '%Y-%m-%d')
            users.append(row)
    return users

def should_notify(user):
    days_since_last_visit = (datetime.today() - user['Last Visit']).days
    return user['Interaction Score'] < 0.5 and days_since_last_visit > 3

def generate_notification(user):
    notification_types = ['Email', 'SMS']
    notification_type = random.choice(notification_types)
    message_templates = [
        "Hey there! We noticed you haven’t visited in a while. Check out our latest offers just for you!",
        "Come back and enjoy exclusive discounts on your favorite items!",
        "We miss you! See what's new and exciting on our platform today!"
    ]
    message = random.choice(message_templates)
    return [user['User ID'], notification_type, message]

def generate_notifications(filename):
    users = load_user_data(filename)
    notifications = [generate_notification(user) for user in users if should_notify(user)]
    return notifications

def save_notifications(filename, notifications):
    header = ["User ID", "Notification Type", "Message"]
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(notifications)

def main():
    num_entries = 100  # Change this to generate more or fewer entries
    activity_filename = "user_activity_data.csv"
    notification_filename = "user_notifications.csv"
    
    data = generate_user_activity_data(num_entries)
    save_to_csv(activity_filename, data)
    
    notifications = generate_notifications(activity_filename)
    save_notifications(notification_filename, notifications)
    print(f"Generated {num_entries} user activity records and saved to {activity_filename}.")
    print(f"Generated {len(notifications)} notifications and saved to {notification_filename}.")

if __name__ == "__main__":
    main()


Generated 100 user activity records and saved to user_activity_data.csv.
Generated 56 notifications and saved to user_notifications.csv.


In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

def load_data(activity_file, notification_file):
    activity_df = pd.read_csv(activity_file)
    notification_df = pd.read_csv(notification_file)
    
    activity_df['Last Visit'] = pd.to_datetime(activity_df['Last Visit'])
    activity_df['Days Since Last Visit'] = (datetime.today() - activity_df['Last Visit']).dt.days
    
    return activity_df, notification_df

def train_notification_model(data):
    X = data[['Days Since Last Visit', 'Time Spent (minutes)', 'Pages Viewed', 'Interaction Score']]
    y = (data['Interaction Score'] < 0.5) & (data['Days Since Last Visit'] > 3)  # Users at risk
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print("Notification Model Accuracy:", accuracy_score(y_test, y_pred))
    
    return model

def train_message_generator(notification_df):
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    
    messages = notification_df['Message'].tolist()
    inputs = ["generate notification message: " + msg for msg in messages]
    
    tokenized_inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
    labels = tokenizer(messages, return_tensors="pt", padding=True, truncation=True).input_ids
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    
    model.train()
    for epoch in range(14):
        optimizer.zero_grad()
        outputs = model(**tokenized_inputs, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    
    return model, tokenizer

def generate_custom_message(name, time_spent, pages_viewed, interaction_score):
    """Create personalized messages based on user behavior"""
    
    if interaction_score < 0.3:
        behavior_msg = "We haven’t seen you engage much. Try exploring more!"
    elif interaction_score < 0.5:
        behavior_msg = "Your last session was short. Let’s make it better next time!"
    else:
        behavior_msg = "You seem to enjoy our content! Discover even more!"
    
    if time_spent < 5:
        time_msg = "You were here briefly last time, come and check out what's new!"
    elif time_spent < 15:
        time_msg = "You spent some time with us. Let's dive deeper into your favorite content!"
    else:
        time_msg = "We love seeing you here! There’s more exciting stuff waiting for you!"
    
    if pages_viewed < 3:
        page_msg = "There are many features you haven't seen yet!"
    elif pages_viewed < 7:
        page_msg = "You explored quite a bit! Here's more you might love."
    else:
        page_msg = "You’re a power user! Let’s take it to the next level!"
    
    return f"Hey {name}, {behavior_msg} {time_msg} {page_msg}"

def generate_notifications(notification_model, message_model, tokenizer, data):
    X = data[['Days Since Last Visit', 'Time Spent (minutes)', 'Pages Viewed', 'Interaction Score']]
    predictions = notification_model.predict(X)
    
    final_output = []
    
    for i, send_notification in enumerate(predictions):
        if send_notification:
            user_id = data.iloc[i]['User ID']
            name = data.iloc[i].get('Name', f"User {user_id}")  # Use 'User {ID}' if Name is missing
            time_spent = data.iloc[i]['Time Spent (minutes)']
            pages_viewed = data.iloc[i]['Pages Viewed']
            interaction_score = data.iloc[i]['Interaction Score']
            
            notification_type = np.random.choice(["Email", "SMS"])  # Random selection
            
            message = generate_custom_message(name, time_spent, pages_viewed, interaction_score)
            
            final_output.append((user_id, notification_type, message))
    
    return final_output

def main():
    activity_file = "user_activity_data.csv"
    notification_file = "user_notifications.csv"
    
    activity_df, notification_df = load_data(activity_file, notification_file)
    notification_model = train_notification_model(activity_df)
    message_model, tokenizer = train_message_generator(notification_df)
    
    notifications = generate_notifications(notification_model, message_model, tokenizer, activity_df)
    
    print("\nGenerated Notifications:")
    for user_id, notif_type, message in notifications:
        print(f"User {user_id} - {notif_type}: {message}")

if __name__ == "__main__":
    main()


Notification Model Accuracy: 0.95
Epoch 1, Loss: 4.838974952697754
Epoch 2, Loss: 4.195798873901367
Epoch 3, Loss: 3.8200137615203857
Epoch 4, Loss: 3.45023512840271
Epoch 5, Loss: 2.772465229034424
Epoch 6, Loss: 2.5105414390563965
Epoch 7, Loss: 2.153144598007202
Epoch 8, Loss: 1.8366410732269287
Epoch 9, Loss: 1.4667432308197021
Epoch 10, Loss: 1.4117679595947266
Epoch 11, Loss: 1.2643442153930664
Epoch 12, Loss: 1.1274261474609375
Epoch 13, Loss: 1.045715093612671
Epoch 14, Loss: 1.0665174722671509

Generated Notifications:
User 65618 - Email: Hey User 65618, Your last session was short. Let’s make it better next time! We love seeing you here! There’s more exciting stuff waiting for you! There are many features you haven't seen yet!
User 28543 - SMS: Hey User 28543, We haven’t seen you engage much. Try exploring more! We love seeing you here! There’s more exciting stuff waiting for you! You’re a power user! Let’s take it to the next level!
User 27279 - Email: Hey User 27279, We hav

In [27]:
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pretrained T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def generate_message(user_id, product_id, visit_count, last_visited):
    """Generates a personalized notification message using T5."""
    prompt = (f"User {user_id} visited Product {product_id} {visit_count} times, last visit on {last_visited}. "
              "Generate a personalized retention message.")
    
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids, max_length=50)
    message = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return message

# Load user activity dataset
user_activity_df = pd.read_csv("user_interactions.csv")

# Generate notifications
notifications = []
for _, row in user_activity_df.iterrows():
    user_id = row['user_id']
    product_id = row['product_id']
    visit_count = row['visit_count']
    last_visited = row['last_visited']
    
    message = generate_message(user_id, product_id, visit_count, last_visited)
    notifications.append([user_id, "Email", message])  # Default to Email

# Save to CSV
notifications_df = pd.DataFrame(notifications, columns=["user_id", "notification_type", "message"])
notifications_df.to_csv("generated_notifications.csv", index=False)

print("Notifications generated and saved to generated_notifications.csv!")


Notifications generated and saved to generated_notifications.csv!


In [23]:
user_activity_df = pd.read_csv("user_interactions.csv")


In [25]:
user_activity_df

,id,user_id,product_id,page_url,interaction_type,visit_count,last_visited,created_at,interaction_score,notification_type,message
0,1,1004,203,/product/205,purchase,10,2025-03-23 15:16:59,2025-03-23 15:16:59,1.00,NaN,NaN
1,2,1002,204,/product/204,add_to_cart,1,2025-03-07 15:16:59,2025-03-23 15:16:59,0.09,Email,Hey User 1002! We noticed you haven’t visited ...
2,3,1001,202,/product/203,click,2,2025-03-17 15:16:59,2025-03-23 15:16:59,0.19,Push,Hey User 1001! We noticed you haven’t visited ...
3,4,1002,204,/product/202,add_to_cart,5,2025-03-12 15:16:59,2025-03-23 15:16:59,0.42,SMS,Hey User 1002! We noticed you haven’t visited ...
4,5,1004,205,/product/202,add_to_cart,8,2025-03-06 15:16:59,2025-03-23 15:16:59,0.65,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
495,496,1004,202,/product/201,view,6,2025-03-20 15:16:59,2025-03-23 15:16:59,0.53,NaN,NaN
496,497,1003,204,/product/203,purchase,2,2025-03-03 15:16:59,2025-03-23 15:16:59,0.17,Email,Hey User 1003! We noticed you haven’t visited ...
497,498,1005,204,/product/205,click,5,2025-03-22 15:16:59,2025-03-23 15:16:59,0.50,NaN,NaN
498,499,1005,205,/product/203,click,6,2025-03-17 15:16:59,2025-03-23 15:16:59,0.51,NaN,NaN
